In [ ]:
!pip install tensorflow-datasets
!pip install tensorflow --upgrade


In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np


(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()


x_train_cnn = x_train.astype('float32') / 255.0
x_test_cnn = x_test.astype('float32') / 255.0


y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)


classes = ["airplane", "automobile", "bird", "cat", "deer",
           "dog", "frog", "horse", "ship", "truck"]


def create_custom_cnn():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model


def create_finetuned_resnet():
    base_model = ResNet50(weights='imagenet', include_top=False,
                        input_shape=(224, 224, 3))
    base_model.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model


def preprocess_resnet(image, label):
    image = tf.image.resize(image, [224, 224])
    image = preprocess_input(image)
    return image, label


batch_size = 64
epochs = 10


train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = (train_dataset
                .map(preprocess_resnet, num_parallel_calls=tf.data.AUTOTUNE)
                .shuffle(buffer_size=1024)
                .batch(batch_size)
                .prefetch(tf.data.AUTOTUNE))

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = (test_dataset
               .map(preprocess_resnet, num_parallel_calls=tf.data.AUTOTUNE)
               .batch(batch_size)
               .prefetch(tf.data.AUTOTUNE))


print("Training Custom CNN...")
cnn_model = create_custom_cnn()
cnn_model.fit(x_train_cnn, y_train,
             epochs=epochs,
             batch_size=batch_size,
             validation_data=(x_test_cnn, y_test))


cnn_pred = np.argmax(cnn_model.predict(x_test_cnn), axis=1)


print("Training ResNet-50...")
resnet_model = create_finetuned_resnet()
resnet_model.fit(train_dataset,
                epochs=epochs,
                validation_data=test_dataset)


resnet_pred = []
for images, _ in test_dataset:
    preds = np.argmax(resnet_model.predict(images), axis=1)
    resnet_pred.extend(preds)

resnet_pred = np.array(resnet_pred)


def print_metrics(y_true, y_pred, model_name):
    print(f"{model_name} Performance:")
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=classes))

    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")

# Compare models
print_metrics(y_test, cnn_pred, "Custom CNN")
print_metrics(y_test, resnet_pred, "Fine-Tuned ResNet-50")

Training Custom CNN...
Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.3203 - loss: 1.8276 - val_accuracy: 0.5537 - val_loss: 1.2547
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.5638 - loss: 1.2328 - val_accuracy: 0.5830 - val_loss: 1.1555
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6343 - loss: 1.0360 - val_accuracy: 0.6427 - val_loss: 1.0036
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.6826 - loss: 0.9183 - val_accuracy: 0.6642 - val_loss: 0.9768
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7072 - loss: 0.8336 - val_accuracy: 0.6653 - val_loss: 0.9879
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7331 - loss: 0.7718 - val_accuracy: 0.7029 - val_loss: 0.8813
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.7533 - loss: 0.7129 - val_accuracy: 0.7082 - val_loss: 0.8640
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7722 - loss: 0.